In [6]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV

In [7]:
shot_chart_url = 'http://stats.nba.com/stats/shotchartdetail?Period=0&VsConference=&LeagueID=00&LastNGames=0&TeamID=0&Position=&Location=&Outcome=&ContextMeasure=FGA&DateFrom=&StartPeriod=&DateTo=&OpponentTeamID=0&ContextFilter=&RangeType=&Season=2015-16&AheadBehind=&PlayerID=201939&EndRange=&VsDivision=&PointDiff=&RookieYear=&GameSegment=&Month=0&ClutchTime=&StartRange=&EndPeriod=&SeasonType=Regular+Season&SeasonSegment=&GameID='
# Get the webpage containing the data
response = requests.get(shot_chart_url)
print response
# Grab the headers to be used as column headers for our DataFrame
headers = response.json()['resultSets'][0]['headers']
print headers
# Grab the shot chart data
shots = response.json()['resultSets'][0]['rowSet']

<Response [200]>
[u'GRID_TYPE', u'GAME_ID', u'GAME_EVENT_ID', u'PLAYER_ID', u'PLAYER_NAME', u'TEAM_ID', u'TEAM_NAME', u'PERIOD', u'MINUTES_REMAINING', u'SECONDS_REMAINING', u'EVENT_TYPE', u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', u'SHOT_ZONE_AREA', u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y', u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG']


In [8]:
shot_df = pd.DataFrame(shots, columns=headers)

In [35]:
def create_dataFrame(shot_chart_url):
    response = requests.get(shot_chart_url)
    # Grab the headers to be used as column headers for our DataFrame
    headers = response.json()['resultSets'][0]['headers']
    # Grab the shot chart data
    shots = response.json()['resultSets'][0]['rowSet']
    return pd.DataFrame(shots, columns=headers)
    

In [37]:
shot_df = create_dataFrame(shot_chart_url)
shot_df.columns

Index([u'GRID_TYPE', u'GAME_ID', u'GAME_EVENT_ID', u'PLAYER_ID',
       u'PLAYER_NAME', u'TEAM_ID', u'TEAM_NAME', u'PERIOD',
       u'MINUTES_REMAINING', u'SECONDS_REMAINING', u'EVENT_TYPE',
       u'ACTION_TYPE', u'SHOT_TYPE', u'SHOT_ZONE_BASIC', u'SHOT_ZONE_AREA',
       u'SHOT_ZONE_RANGE', u'SHOT_DISTANCE', u'LOC_X', u'LOC_Y',
       u'SHOT_ATTEMPTED_FLAG', u'SHOT_MADE_FLAG'],
      dtype='object')

In [25]:
def transform(data):
    action_type = pd.get_dummies(data.ACTION_TYPE)
    period = pd.get_dummies(data.PERIOD)
    shot_type = pd.get_dummies(data.SHOT_TYPE)
    shot_zone_basic = pd.get_dummies(data.SHOT_ZONE_BASIC)
    shot_zone_area = pd.get_dummies(data.SHOT_ZONE_AREA)
    shot_zone_range = pd.get_dummies(data.SHOT_ZONE_RANGE)

    new_shot_chart = pd.concat([action_type, period, shot_type, shot_zone_basic, shot_zone_area, shot_zone_range, data.SHOT_MADE_FLAG], axis=1)
    return new_shot_chart

In [28]:
new_shot_chart= transform(shot_df)

In [29]:
logistic = LogisticRegression()
def train_test_splitter(model, X, y, train_size=0.5):
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size)
    model.fit(X_train, y_train)
    return X_train, X_test, y_train, y_test, model

In [30]:
predictors = predictors = [   'Alley Oop Layup shot',
           'Cutting Finger Roll Layup Shot',
                       'Cutting Layup Shot',
                        'Driving Bank shot',
                        'Driving Dunk Shot',
           'Driving Finger Roll Layup Shot',
          'Driving Floating Bank Jump Shot',
               'Driving Floating Jump Shot',
                        'Driving Hook Shot',
                       'Driving Layup Shot',
               'Driving Reverse Layup Shot',
                       'Fadeaway Bank shot',
                       'Fadeaway Jump Shot',
                   'Finger Roll Layup Shot',
                       'Floating Jump shot',
                                'Hook Shot',
                           'Jump Bank Shot',
                                'Jump Shot',
                               'Layup Shot',
                         'Pullup Bank shot',
                         'Pullup Jump shot',
                       'Putback Layup Shot',
                       'Reverse Layup Shot',
                        'Running Dunk Shot',
           'Running Finger Roll Layup Shot',
                        'Running Jump Shot',
                       'Running Layup Shot',
                'Running Pull-Up Jump Shot',
               'Running Reverse Layup Shot',
                      'Step Back Jump shot',
                           'Tip Layup Shot',
                     'Turnaround Bank shot',
       'Turnaround Fadeaway Bank Jump Shot',
                 'Turnaround Fadeaway shot',
                     'Turnaround Jump Shot',
                                           1,
                                           2,
                                           3,
                                           4,
                                           5,
                           '2PT Field Goal',
                           '3PT Field Goal',
                        'Above the Break 3',
                                'Backcourt',
                    'In The Paint (Non-RA)',
                            'Left Corner 3',
                                'Mid-Range',
                          'Restricted Area',
                           'Right Corner 3',
                           'Back Court(BC)',
                                'Center(C)',
                     'Left Side Center(LC)',
                             'Left Side(L)',
                    'Right Side Center(RC)',
                            'Right Side(R)',
                                '16-24 ft.',
                                  '24+ ft.',
                                 '8-16 ft.',
                          'Back Court Shot',
                          'Less Than 8 ft.']

In [82]:
X_train, X_test, y_train, y_test, model = train_test_splitter(logistic, new_shot_chart[predictors], new_shot_chart.SHOT_MADE_FLAG)

model.score(X_test, y_test)

KeyError: "[u'Driving Jump shot' u'Driving Slam Dunk Shot' u'Dunk Shot'\n u'Running Bank shot' u'Running Hook Shot' u'Slam Dunk Shot'\n u'Turnaround Hook Shot'] not in index"

In [45]:
shot_chart_url = 'http://stats.nba.com/stats/shotchartdetail?Period=0&VsConference=&LeagueID=00&LastNGames=0&TeamID=0&Position=&Location=&Outcome=&ContextMeasure=FGA&DateFrom=&StartPeriod=&DateTo=&OpponentTeamID=0&ContextFilter=&RangeType=&Season=2014-15&AheadBehind=&PlayerID=201939&EndRange=&VsDivision=&PointDiff=&RookieYear=&GameSegment=&Month=0&ClutchTime=&StartRange=&EndPeriod=&SeasonType=Regular+Season&SeasonSegment=&GameID='
shot_df_2014 = create_dataFrame(shot_chart_url)
shot_df_2014
new_shot_chart_2014= transform(shot_df_2014)
new_shot_chart_2014.columns
predictors = [             'Driving Bank shot', 'Driving Finger Roll Layup Shot',
                    'Driving Jump shot',             'Driving Layup Shot',
           'Driving Reverse Layup Shot',         'Driving Slam Dunk Shot',
                            'Dunk Shot',             'Fadeaway Bank shot',
                   'Fadeaway Jump Shot',         'Finger Roll Layup Shot',
                   'Floating Jump shot',                 'Jump Bank Shot',
                            'Jump Shot',                     'Layup Shot',
                     'Pullup Bank shot',               'Pullup Jump shot',
                   'Putback Layup Shot',             'Reverse Layup Shot',
                    'Running Bank shot', 'Running Finger Roll Layup Shot',
                    'Running Hook Shot',              'Running Jump Shot',
                   'Running Layup Shot',     'Running Reverse Layup Shot',
                       'Slam Dunk Shot',            'Step Back Jump shot',
                 'Turnaround Bank shot',       'Turnaround Fadeaway shot',
                 'Turnaround Hook Shot',           'Turnaround Jump Shot',
                                       1,                                 2,
                                       3,                                 4,
                                       5,                 '2PT Field Goal',
                       '3PT Field Goal',              'Above the Break 3',
                            'Backcourt',          'In The Paint (Non-RA)',
                        'Left Corner 3',                      'Mid-Range',
                      'Restricted Area',                 'Right Corner 3',
                       'Back Court(BC)',                      'Center(C)',
                 'Left Side Center(LC)',                   'Left Side(L)',
                'Right Side Center(RC)',                  'Right Side(R)',
                            '16-24 ft.',                        '24+ ft.',
                             '8-16 ft.',                'Back Court Shot',
                      'Less Than 8 ft.']
X_train, X_test, y_train, y_test, model = train_test_splitter(logistic, new_shot_chart_2014[predictors], new_shot_chart_2014.SHOT_MADE_FLAG)

model.score(X_test, y_test)

0.64232488822652756

In [55]:
combined = pd.concat([shot_df, shot_df_2014])
new_shot_chart_combined= transform(combined)
predictors = new_shot_chart_combined.columns[:-1]

In [79]:
X_train, X_test, y_train, y_test, model = train_test_splitter(logistic, new_shot_chart_combined[predictors], new_shot_chart_combined.SHOT_MADE_FLAG)

model.score(X_test, y_test)

0.6604850213980028